In [12]:
import os
from dotenv import load_dotenv

## Pinecone Vector Store
from pinecone import Pinecone, ServerlessSpec

## embeddings
from langchain_openai import OpenAIEmbeddings

## Pinecone Vector Store
from langchain_pinecone import PineconeVectorStore
from langchain_core.documents import Document

load_dotenv()

## Pinecone config
api_key = os.getenv("PINECONE_API_KEY")
index_name = os.getenv("PINECONE_INDEX_NAME")

## Set environment variables
os.environ["OPENAI_API_KEY"]  = os.getenv("OPENAI_API_KEY")

In [2]:
pc=Pinecone(api_key=api_key)

In [5]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small",dimensions=1024)

In [8]:
if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=1024,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

index = pc.Index(index_name)

In [10]:
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [13]:
document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]

In [14]:
vector_store.add_documents(documents=documents)

['345aed1c-8f8f-4de7-9ecf-840b5bed9cc5',
 '3bc4b2cb-c7cf-4de0-b2db-46f83d36470f',
 '1823d3a9-3050-4f60-bfe5-86ebd8338284',
 'ae4e1137-f5f9-400f-82d0-207ed26b93aa',
 '431b3e70-49f5-47a4-b9c2-3cabe1779cf6',
 '89677ead-1053-4197-a34b-b6939b0ff971',
 '80c94be6-0bd6-4c8d-9ae0-f69b240b5d09',
 '46098313-303c-41b8-86c2-91a40ac744e6',
 'c607995a-f02e-4b10-a165-5924a246b302',
 'cd2af717-530f-452e-b563-be5870d2b09d']

In [15]:
### Query Directly
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2,
    filter={"source": "tweet"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

In [16]:
results = vector_store.similarity_search_with_score(
    "Will it be hot tomorrow?", k=1, filter={"source": "news"}
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

In [17]:
### Retriever
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 1, "score_threshold": 0.4},
)
retriever.invoke("Stealing from the bank is a crime", filter={"source": "news"})

[Document(id='ae4e1137-f5f9-400f-82d0-207ed26b93aa', metadata={'source': 'news'}, page_content='Robbers broke into the city bank and stole $1 million in cash.')]